In [ ]:
import bs4 as bs
import pickle 
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import time

In [ ]:
def tv():
    url = 'https://www.tradingview.com/markets/stocks-usa/market-movers-gainers/'
    resp = requests.get(url)
    soup = bs.BeautifulSoup(resp.text)
    table = soup.find('table', {'tv-data-table tv-screener-table'})

    tickers = []
    lasts = []
    change_percents = []
    changes = []
    ratings = []
    volumes = []
    mkt_caps = []
    price_earnings = []
    eps = []
    employees = []
    sectors = []

    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        last = row.findAll('td')[1].text
        change_percent = row.findAll('td')[2].text
        change = row.findAll('td')[3].text
        rating = row.findAll('td')[4].text
        volume = row.findAll('td')[5].text
        mkt_cap = row.findAll('td')[6].text
        price_earning = row.findAll('td')[7].text
        Eps = row.findAll('td')[8].text
        employee = row.findAll('td')[9].text
        sector = row.findAll('td')[10].text



        tickers.append(ticker)
        lasts.append(last)
        change_percents.append(change_percent)
        changes.append(change)
        ratings.append(rating)
        volumes.append(volume)
        mkt_caps.append(mkt_cap)
        price_earnings.append(price_earning)
        eps.append(Eps)
        employees.append(employee)
        sectors.append(sector)

    stocks = []

    for ticker in tickers:
        symbol = ''
        counter = 0
        for letter in ticker[3:]:
            counter+=1
            if letter.isupper():
                symbol += letter
            if counter > 6:
                break
        stocks.append(symbol)

    business = []

    for ticker in tickers:
        name = ''
        for letter in ticker:
            if letter.isupper() or letter==' ':
                name+=letter
        business.append(name)

    trading_view = pd.DataFrame({
        'RANK':range(1,len(stocks)+1),
        'DATE':[dt.datetime.now().strftime('%d/%m/%y')]*len(tickers),
        'TIME':[dt.datetime.now().strftime('%I:%M%p ')]*len(tickers),
        'TICKER':stocks,
        'NAME':business,
        'LAST':lasts,
        'CHG%':change_percents,
        'CHG$':changes,
        'RATING':ratings,
        'VOLUME':volumes,
        'MKT CAP':mkt_caps,
        'P/E':price_earnings,
        'EPS':eps,
        'EMPLOYEES':employees,
        'SECTOR':sectors    
    })

    #Remove % sign
    trading_view['CHG%'] = trading_view['CHG%'].str[:-1]

    #Change dtype to float
    trading_view[['LAST','CHG%','CHG$']] = trading_view[['LAST','CHG%','CHG$']].astype(float) 

    #Set RANK as index
    #trading_view.set_index('RANK', inplace=True)

    #Create CSV for every day
    name = r'C:\Users\19258\Finance\Morning_Prices\Prices_'+str(dt.datetime.now().strftime('%d_%m_%y'))+'.csv'
    trading_view.to_csv(name,mode='a')
    
    return trading_view

In [ ]:
tv().head()

,RANK,DATE,TIME,TICKER,NAME,LAST,CHG%,CHG$,RATING,VOLUME,MKT CAP,P/E,EPS,EMPLOYEES,SECTOR
0,1,19/03/21,07:56PM,HOFV,HHOFVHALL OF FAME RESORT ENTMT COMPANY,4.27,56.41,1.54,Buy,175.560M,355.625M,—,-1.33,30.00,Consumer Services
1,2,19/03/21,07:56PM,SPCB,SSPCBSUPERCOM LTD NEW,2.19,51.38,0.74,Buy,141.874M,41.241M,—,—,—,Electronic Technology
2,3,19/03/21,07:56PM,CLVS,CLVSCLOVIS ONCOLOGY INC,7.72,45.11,2.40,Buy,312.723M,806.972M,—,-4.45,429.00,Health Technology
3,4,19/03/21,07:56PM,SNES,SSNESSENESTECH INC,2.38,41.37,0.69,Buy,73.834M,20.112M,—,-4.82,38.00,Health Technology
4,5,19/03/21,07:56PM,MRKR,MMRKRMARKER THERAPEUTICS INC,2.52,39.72,0.71,Buy,61.323M,127.589M,—,-0.61,44.00,Consumer Services


In [ ]:
#Automate the function
'''
Need these to set up start and end times
'''
current_time = dt.datetime.now()
year = int(current_time.strftime('%Y'))
month = int(current_time.strftime('%m'))
day = int(current_time.strftime('%d'))


#start time 6:30AM of current day
start = dt.datetime(year,month,day,hour=6, minute=49).strftime('%H:%M%p %d/%m/%Y')
#End time 1:00PM of current day
end = dt.datetime(year,month,day,hour=13, minute=0).strftime('%H:%M%p %d/%m/%Y')

while True:
    now = dt.datetime.now().strftime('%H:%M%p %d/%m/%Y') #Update current time after each iteration
    if now > start and now < end:
        tv() #Collect data
        time.sleep(300) #15 min
    if now >= end:
        break